# Corona Project


In [1]:
import numpy as np 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import json
import plotly.express as px
from bokeh.palettes import RdYlBu11 as palette
from bokeh.palettes import Spectral6, brewer
from bokeh.plotting import figure, output_file, show, save
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook
from bokeh.layouts import column, row, widgetbox
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)

In [2]:
#this pulls the geojson from my github and transforms it into readable data
url = "https://raw.githubusercontent.com/oczerwia/corona_map/main/landkreise_simplify20.geojson"
NRW_map =  requests.get(url)
NRW_map = NRW_map.text
NRW_data = json.loads(NRW_map)
geo_df = gpd.read_file(url)
geo_df.crs = {'init': 'epsg:4326'}

In [3]:
RKI_url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=BL%20%3D%20%27NORDRHEIN-WESTFALEN%27&outFields=OBJECTID,ADE,GF,BSG,RS,AGS,SDV_RS,GEN,BEZ,IBZ,BEM,NBD,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,RS_0,AGS_0,WSK,EWZ,KFL,DEBKG_ID,death_rate,cases,deaths,cases_per_100k,cases_per_population,BL,BL_ID,county,last_update,cases7_per_100k,recovered,EWZ_BL,cases7_bl_per_100k&outSR=4326&f=json"
RKI = requests.get(RKI_url)
RKI = RKI.text
RKI_data = json.loads(RKI)

In [4]:
RKI_dataframe = pd.DataFrame(columns = ["Name", "cases","cases_per_100k","deaths"])
RKI_dataframe["Name"] = [name["attributes"]["GEN"] for name in RKI_data["features"]]
RKI_dataframe["cases"] = [name["attributes"]["cases"] for name in RKI_data["features"]]
RKI_dataframe["cases_per_100k"] = [name["attributes"]["cases7_per_100k"] for name in RKI_data["features"]]
RKI_dataframe["deaths"] = [name["attributes"]["deaths"] for name in RKI_data["features"]]

In [5]:
population_df = pd.DataFrame(columns = ["Name", "population"])
population_df["Name"] = [cityname["properties"]["GEN"] for cityname in NRW_data["features"]]
population_df["population"] = [population["properties"]["destatis"]["population"] for population in NRW_data["features"]]

In [6]:
full_stats = RKI_dataframe.merge(population_df, how= "inner",on ="Name")
full_stats.head()

,Name,cases,cases_per_100k,deaths,population
0,Düsseldorf,9412,134.431728,89,619294
1,Duisburg,9961,240.632382,148,498590
2,Essen,7485,144.999657,81,583109
3,Krefeld,3380,189.519693,36,227020
4,Mönchengladbach,3312,131.017415,66,261454


In [7]:
geo_df = geo_df.rename(columns={'geometry': 'geometry','GEN':'Name'}).set_geometry('geometry')
geo_df.drop(columns = ["ADE", "GF", "BSG", "RS", "AGS", "SDV_RS","BEZ", "IBZ", "BEM", "SN_V2", "SN_G", "FK_S3", "NUTS",
                       "RS_0", "AGS_0", "WSK", "DEBKG_ID","NBD", "SN_L", "SN_R", "SN_K", "SN_V1", "destatis"], inplace = True)

In [8]:
full_stats = geo_df.merge(full_stats, how = "inner", on = "Name")

In [9]:
full_stats.head()
full_stats.sort_values(by = "cases_per_100k", ascending = False)[:100]

,Name,geometry,cases,cases_per_100k,deaths,population
38,Lippe,"POLYGON ((8.99588 52.18590, 8.99627 52.18565, ...",4189,263.874261,46,348391
44,Hamm,"POLYGON ((7.83359 51.74116, 7.83263 51.73622, ...",3199,261.788835,52,179111
43,Hagen,"POLYGON ((7.48718 51.41757, 7.48867 51.41708, ...",3251,252.800950,41,188814
8,Solingen,"POLYGON ((7.08315 51.22041, 7.08690 51.21995, ...",2840,248.673428,31,159360
1,Duisburg,"POLYGON ((6.70300 51.55403, 6.70729 51.55077, ...",9961,240.632382,148,498590
45,Herne,"POLYGON ((7.24968 51.56589, 7.24965 51.56470, ...",2987,239.694725,23,156374
26,Bottrop,"POLYGON ((6.94395 51.63753, 6.94782 51.63461, ...",1571,209.245949,11,117383
42,Dortmund,"POLYGON ((7.42328 51.59011, 7.42053 51.58669, ...",8310,205.524862,58,587010
35,Gütersloh,"POLYGON ((8.28586 52.13111, 8.28587 52.12915, ...",6573,205.514361,40,364083
31,Recklinghausen,"POLYGON ((7.18143 51.82077, 7.18613 51.82100, ...",9022,205.165948,85,615261


In [10]:
full_stats = full_stats.to_crs({'init': 'epsg:4326'})

C:\Users\olive\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
copy_stats = full_stats.copy()

In [12]:
copy_stats.to_crs(epsg=3035, inplace = True)
geosource = GeoJSONDataSource(geojson = copy_stats.to_json())
geoprint = json.loads(copy_stats.to_json())

In [13]:
p = figure(title = 'NRW_map', 
           plot_height = 600 ,
           plot_width = 600, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

kreise = p.patches('xs','ys', source = geosource,
                   fill_color = "blue",
                   line_color = "white", 
                   line_width = 0.25, 
                   fill_alpha = 1)

p.add_tools(HoverTool(renderers = [kreise],
                      tooltips = [('Name','@Name'), ("Einwohner", "@population"), 
                                  ("Cases/100k", "@cases_per_100k"), ("Deaths", "@deaths")]))


output_notebook()

Loading BokehJS ...

In [14]:
#palette = brewer['BuGn'][8]
palette = ['#fee5d9','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d']
#palette = palette[::-1]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 300)

tick_labels = {"0":"0", }

color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 600, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = "horizontal",
                     major_label_overrides = tick_labels)

states = p.patches("xs","ys", source = geosource,
                   fill_color = {"field" :'cases_per_100k',
                                 "transform" : color_mapper},
                   line_color = "black", 
                   line_width = 0.4 , 
                   fill_alpha = 1)

p.add_layout(color_bar, "below")

In [15]:
show(p)

In [16]:
#this is the basis for an automated updater
#the second script in this repo will automatically update the dataframe daily to make further analysis possible
copy_stats.to_csv(index = False, path_or_buf =r"C:\Users\olive\Kaggle\Corona_map\corona_map\first_data.csv")